In [1]:
WORKDIR=%pwd 

# Example 2: Water dipole and polarizability
## Tasks
 - Train dipole and polarizability of [bulk water](https://github.com/lab-cosmo/SA-GPR/tree/master/example/water_bulk).
 - Use LAMMPS Calculator interface.
 - Plot the infrared and raman spectrum. 

## Prepare data 
You can see the following files required by training in this fold.

```bash
water
|- dipole.yaml               
|- polar.yaml
|- data/
   |- train.xyz            
   |- test.xyz             
```

[`dipole.yaml`](dipole.yaml) and [`polar.yaml`](polar.yaml) controls the details of model architecture and training for dipole and polarizability respectively. Some import parameters in this tasks are:

For `dipole.yaml`:
```yaml
cutoff: 4.0
Data:
  trainSet: data/train.xyz
  testSet: data/test.xyz
Train:
  targetProp: ["dipole"] 
  weight: [1.0]
```

These parameters mean the cutoff in this task is 4.0 Å. We use `data/train.xyz` as trainset and `data/test.xyz` as testset (actually it should be vaildation dataset here). We only train `dipole` in this task.

For `polar.yaml`:
```yaml
Train:
  targetProp: ["polarizability"] 
```
means we only train `polarizability`.

## Train
Now, we can train the model. For dipole:

In [8]:
# ! hotpp train -i dipole.yaml -o dipole --load_checkpoint dipole.ckpt -ll INFO INFO
! hotpp train -i dipole.yaml -o dipole -ll INFO INFO

15:45:02   

    )            (    (     
 ( /(          ) )\ ) )\ )  
 )\())      ( /((()/((()/(  
((_)\   (   )\())/(_))/(_)) 
 _((_)  )\ (_))/(_)) (_))   
| || | ((_)| |_ | _ \| _ \  
| __ |/ _ \|  _||  _/|  _/  
|_||_|\___/ \__||_|  |_|    
HotPP (v.1.0.0 RELEASE)

Traceback (most recent call last):
  File "/home/dbissuel/Documents/ILM/TheoChem/Claire/Guillaume/beta_ai/hotpp/.venv/bin/hotpp", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/dbissuel/Documents/ILM/TheoChem/Claire/Guillaume/beta_ai/hotpp/hotpp/entrypoints/main.py", line 236, in main
    f(**dict_args)
  File "/home/dbissuel/Documents/ILM/TheoChem/Claire/Guillaume/beta_ai/hotpp/hotpp/entrypoints/train.py", line 425, in main
    with open(input_file) as f:
         ^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'dipole.yaml'


where 
- `-i dipole.yaml` means we use `dipole.yaml` as input parameters (default is `input.yaml`)
- `-o dipole` means we use `dipole` as the output folder (default is `outDir`)
- `-ll INFO INFO` means the log level for stream and file outputs are `INFO` and `INFO` (default are `DEBUG` and `INFO`)
and similarly, for polarizability:

In [4]:
# ! hotpp train -i polar.yaml -o polar --load_checkpoint polar.ckpt -ll INFO INFO
! hotpp train -i polar.yaml -o polar -ll INFO INFO

15:25:51   

    )            (    (     
 ( /(          ) )\ ) )\ )  
 )\())      ( /((()/((()/(  
((_)\   (   )\())/(_))/(_)) 
 _((_)  )\ (_))/(_)) (_))   
| || | ((_)| |_ | _ \| _ \  
| __ |/ _ \|  _||  _/|  _/  
|_||_|\___/ \__||_|  |_|    
HotPP (v.1.0.0 RELEASE)

15:25:53   Using seed 0
15:25:53   Preparing data...
15:25:54   n_neighbor   : 25.826607142857142
15:25:54   all_elements : [1, 8]
15:25:54   ground_energy  : [0.0]
15:25:54   std   : 1.0
15:25:54   mean  : 0.7
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
You are using a CUDA device ('NVIDIA RTX 4000 Ada Generation Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable


## Eval
After training done, we can evaluate the models:

In [5]:
%mkdir eval
%cd eval
! hotpp eval -m ../dipole/best.pt -d ../data/test.xyz -p dipole --device cuda -b 16
! hotpp eval -m ../polar/best.pt -d ../data/test.xyz -p polarizability --device cuda -b 16

/home/dbissuel/Documents/ILM/TheoChem/Claire/Guillaume/beta_ai/hotpp/examples/water/eval
/home/dbissuel/Documents/ILM/TheoChem/Claire/Guillaume/beta_ai/hotpp/hotpp/entrypoints/eval.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on G

And you can analyze them. To plot them to compare with DFT:

In [6]:
! hotpp plot -p polarizability dipole

polarizability: 0.6400 0.4800
   dipole   : 0.4567 0.3472


This command means that we plot `polarizability` and `dipole` calculated by `HotPP` and the target values. And you can see the results:  

|`polarizability`|`dipole`|
|:-:|:-:|
|<img src="eval/polarizability.png" width = "300"  alt="polarizability" />|<img src="eval/dipole.png" width = "300"  alt="dipole" />|


If you need plot `peratom polarizability` instead of `polarizability`, just use:
```bash
$ hotpp plot -p per_polarizability
```

## Molecule Dynamics
Then we introduce how to calculate infrared and raman spectrum with lammps. 
First, we freeze the model so you can use it without `hotpp` being installed.

In [24]:
%cd {WORKDIR}
%mkdir lmps
%cd lmps
! hotpp freeze ../dipole/best.pt -o dipole.pt
! hotpp freeze ../polar/best.pt -o polar.pt

/home/gegejun/src/hotpp/examples/water
mkdir: cannot create directory ‘lmps’: File exists
/home/gegejun/src/hotpp/examples/water/lmps


We can get `ase-dipole.pt`, `ase-polar.pt`, `lammps-dipole.pt`, and `lammps-polar.pt`. As its name, `lammps-dipole.pt` and `lammps-polar.pt` are we need. Besides, a machine learning potential is required to run molecular dynamics, and it can be trained as shown in the [carbon] example. Here, we skip this process and directly use the pre-trained force field.
In summary, we need:
```bash
lmps
|- lammps-dipole.pt      # model to calculate dipole
|- lammps-polar.pt       # model to calculate polarizability
|- lammps-infer.pt       # model to calculate energy, forces, and virials
|- restart.xyz           # initial structure
|- in.lammps             # lammps input file
|- lmp_hotpp             # lammps binary compiled with hotpp
```
And the `lmp_hotpp` can be got as shown in the [document](https://hotpp.readthedocs.io/en/latest/install.html).  

To quickly show how we calculate infrared and raman spectrum, in this example we just run a 20 ps NVE with only 96 H<sub>2</sub>O. When used in practice, the simulation time and structure size should be extended.   
Now we can run lammps by:

In [4]:
! ./lmp_hotpp -in in.lammps

LAMMPS (2 Aug 2023 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/lammps-2Aug2023/src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0 0 0) to (13.348493 15.413518 14.532002)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  288 atoms
  reading velocities ...
  288 velocities
  read_data CPU = 0.003 seconds
The simulations are performed on the GPU
ok
Generated 0 of 1 mixed pair_coeff terms from geometric mixing rule
The simulations are performed on the GPU
ok
The simulations are performed on the GPU
ok
Neighbor list info ...
  update: every = 10 steps, delay = 0 steps, check = no
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 6
  ghost atom cutoff = 6
  binsize = 3, bins = 5 6 5
  3 neighbor lists, perpetual/occasional/extra = 1 2 0
  (1) pair miao, perpetual
      attributes: full, newton on
      pair build: full/bin/atomonly
      stencil: full/bin/3d
      bin: stan

Now we have got `dipole.txt` and `polar.txt`, and can calculate IR and Raman by:

In [14]:
%cp ../../../tools/spectrum.py .
import numpy as np
from spectrum import load_lmps_dipole, load_lmps_polar, calc_acf_dp, calc_acf_beta, calc_ir, calc_raman, fs
import matplotlib.pyplot as plt

N = 1000
T = 298.15
dt = 1 * fs
w_max = 4000

dipole = load_lmps_dipole("dipole.txt")
acf_dp = calc_acf_dp(dipole, N)
freq, ir = calc_ir(acf_dp, N, T, dt, w_max)
ir = ir / ir.max()

fig = plt.figure(figsize=(12, 7))
fig.patch.set_facecolor('white')
ax = plt.gca()
ax.spines['bottom'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['top'].set_linewidth(3)
ax.tick_params(labelsize=16)
ax.set_xlabel('Frequency (cm-1)', fontsize=20)
ax.set_ylabel('IR intensity (arb. units)', fontsize=20)
plt.plot(freq, ir, color='blue', linewidth=3)
plt.savefig("ir.png")
plt.close()

polar = load_lmps_polar("polar.txt")
acf_beta = calc_acf_beta(polar, N)
freq, raman = calc_raman(acf_beta, N, T, dt, w_max)
raman /= raman.max()

fig = plt.figure(figsize=(12, 7))
fig.patch.set_facecolor('white')
ax = plt.gca()
ax.spines['bottom'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['top'].set_linewidth(3)
ax.tick_params(labelsize=16)
ax.set_xlabel('Frequency (cm-1)', fontsize=20)
ax.set_ylabel('anisotropic Raman intensity (arb. units)', fontsize=20)
plt.plot(freq, raman, color='blue', linewidth=3)
plt.savefig("raman.png")
plt.close()

And you can see the results:
|`IR`|`Raman`|
|:-:|:-:|
|<img src="lmps/ir.png" width = "500"  alt="IR" />|<img src="lmps/raman.png" width = "500"  alt="Raman" />|

